In [1]:
import pandas as pd
import gc

广告曝光数据：
aid_request 广告请求 唯一标识

request_time 广告请求的时间戳

aid_location 广告位 加密后无业务含义 只能区分不同广告位

uid 用户ID

aid 广告id

aid_size 曝光广告尺寸

bid 曝光广告出价

pctr 曝光率预测

quality_ecpm 质量得分

totalEcpm 广告排序的质量依据

In [2]:
data = pd.read_hdf("totalExposureLog.h5")

KeyboardInterrupt: 

In [ ]:
data.head()

In [3]:
#   静态数据
df =pd.read_csv('ad_static_feature.out', sep='\t', names=['aid','create_timestamp','advertiser','good_id','good_type','ad_type_id','ad_size'])
df=df.fillna(-1)
df.tail()

D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,aid,create_timestamp,advertiser,good_id,good_type,ad_type_id,ad_size
735906,3990,1551428228,4255,3401,18,218,36
735907,207209,1553238282,15898,29999,18,145,-1
735908,160607,1552999465,13469,13819,8,32,-1
735909,687681,1553023368,23225,-1,1,211,-1
735910,222150,1551340613,4250,14527,18,94,36


In [8]:
#静态广告数据处理
for f in ['aid','create_timestamp','advertiser','good_id','good_type','ad_type_id']:
        items=[]
        for item in df[f].values:
            try:
                items.append(int(item))
            except:
                items.append(-1)
        df[f]=items
        df[f]=df[f].astype(int)
df['ad_size']=df['ad_size'].apply(lambda x:' '.join([str(int(float(y))) for y in str(x).split(',')]))    
df.to_pickle('ad_static_feature.pkl')

ValueError: could not convert string to float: '17 57'

In [20]:
df.to_pickle('ad_static_feature.pkl')
del df 
gc.collect()

1319

In [21]:
#   测试数据
df=pd.read_csv('test_sample.dat', sep='\t', names=['id','aid','create_timestamp','ad_size','ad_type_id','good_type','good_id','advertiser','delivery_periods','crowd_direction','bid'])
df=df.fillna(-1)
df[['id','aid','create_timestamp','ad_size','ad_type_id','good_type','good_id','advertiser']]=df[['id','aid','create_timestamp','ad_size','ad_type_id','good_type','good_id','advertiser']].astype(int)
df['bid']=df['bid'].astype(float)
df.to_pickle('test_sample.pkl')

del df
gc.collect()

0

In [4]:
#  用户数据
df =pd.read_csv('user_data', sep='\t', 
              names=['uid','age','gender','area','status','education','concuptionAbility','os','work','connectionType','behavior'])
df=df.fillna(-1)
df.head()

,uid,age,gender,area,status,education,concuptionAbility,os,work,connectionType,behavior
0,624218,753,2,"523,7589,12527,9019,14605,13489,302,6222,14214...",0,6,1,2,0,4,"32612,34406,3413,33059,4275,4667,4561,34051,42..."
1,271475,601,3,"9019,9032,663,12527,14532,2416,2599,1987,9687,...",13,7,2,2,0,2,"4990,4992,3413,33061,4115,4876,2393,34167,3441..."
2,44439,819,3,"12527,14188,9841,9019,14605,6222,9962,6643,172...",13,7,3,2,4,4,"3099,4403,34414,431,34415,5108,31686,3240,3229..."
3,863717,1,2,"12527,3435,2398,11069,1292,9019,14605,14348,62...",6,5,3,2,0,2,"34293,845,3413,3099,33061,33059,4120,4410,4415..."
4,1305469,714,3,"8928,9019,1540,719,12527,6051,10545,12118,7107...",13,5,3,2,0,2,"32612,34289,34406,3413,4275,4410,4561,34051,34..."


In [27]:
print(type(df.iloc[1,1]))

<class 'numpy.int64'>


In [5]:
df[['uid','age','gender','education','consuptionAbility','os','connectionType']]=df[['uid','age','gender','education','concuptionAbility','os','connectionType']].astype(int)
for f in ['area','status','work','behavior']:
    df[f]=df[f].apply(lambda x:' '.join(x.split(',')))
df.to_csv('user_data.csv')
del df
gc.collect()

KeyboardInterrupt: 

In [ ]:
train_df=pd.read_hdf('totalExposureLog.h5')

In [ ]:
train

In [ ]:

train_df['request_day']=train_df['request_timestamp']//(3600*24)
wday=[]
hour=[]
minute=[]
for x in tqdm(train_df['request_timestamp'].values,total=len(train_df)):
    localtime=time.localtime(x)
    wday.append(localtime[6])
    hour.append(localtime[3])
    minute.append(localtime[4])
train_df['wday']=wday
train_df['hour']=hour
train_df['minute']=minute
train_df['period_id']=train_df['hour']*2+train_df['minute']//30
dev_df=train_df[train_df['request_day']==17974]
del dev_df['period_id']
del dev_df['minute']
del dev_df['hour']
log=train_df
tmp = pd.DataFrame(train_df.groupby(['aid','request_day']).size()).reset_index()
tmp.columns=['aid','request_day','imp']
log=log.merge(tmp,on=['aid','request_day'],how='left')
log[log['request_day']<17973].to_pickle('data/user_log_dev.pkl')
log.to_pickle('data/user_log_test.pkl')
del log
del tmp
gc.collect()
del train_df['period_id']
del train_df['minute']
del train_df['hour']    